- prepare data in h5 formant
    - ECoG
    - Spec
    - Waveform
    - onset offset

'ecog_alldataset','label_alldataset',\
'formant_re_alldataset','pitch_re_alldataset','intensity_re_alldataset',\
'start_ind_re_valid_alldataset','end_ind_re_valid_alldataset','wave_re_spec_alldataset',\
'wave_re_alldataset','wave_re_spec_amp_alldataset','noisesample_re_alldataset'

In [12]:
import h5py
import numpy as np
metadata =  h5py.File('../data/NY869.h5')

/ext3/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  after removing the cwd from sys.path.


In [13]:
for key in metadata.keys():
    print (key, metadata[key].shape)

ecog_alldataset (189893, 64)
end_ind_re_valid_alldataset (378,)
formant_re_alldataset (189893, 6)
intensity_re_alldataset (189893,)
label_alldataset (378,)
noisesample_re_alldataset (512,)
pitch_re_alldataset (189893,)
start_ind_re_valid_alldataset (378,)
wave_re_alldataset (24306304, 1)
wave_re_spec_alldataset (189893, 512)
wave_re_spec_amp_alldataset (189893, 512)


- in `configs/train_param_production.ymal` we set ECoG and Spec sampling rate to `125`, and our segment has 128 frames
- preprare your own ECoG signal array, should look like: (189893, 64) which means Temporal length \* electrodes channel
- prepare your onset and offset to segment out the trial, then save them to `start_ind_re_valid_alldataset` and `end_ind_re_valid_alldataset` correspondingly
- prepare the trial's corresponding word label and save to `label_alldataset`
- prepare waveform dataset, sampling rate `16000`, save to `wave_re_alldataset`
- prepare spectrogram dataset, sampling rate `16000` save to `wave_re_spec_alldataset`
```
wave_re_spec = wave2spec(wave),n_fft=N_FFT_num,noise_db=cfg.MODEL.NOISE_DB,max_db=cfg.MODEL.MAX_DB,power=2 if cfg.MODEL.POWER_SYNTH else 1)
wave_re_spec_amp = wave2spec( wave,n_fft=N_FFT_num,noise_db=cfg.MODEL.NOISE_DB,max_db=cfg.MODEL.MAX_DB,to_db=False,power=2 if cfg.MODEL.POWER_SYNTH else 1)
```
- optionally, prepare `pitch_re_alldataset`, `intensity_re_alldataset`, `formant_re_alldataset`
- prepare noise distribution dataset, 
    ```
    baseline_ind_spec = np.concatenate([np.arange((start_ind_valid[i]*1.0/self.DOWN_ECOG_FS*self.DOWN_TF_FS-self.DOWN_TF_FS//4).astype(np.int64),(start_ind_valid[i]*1.0/self.DOWN_ECOG_FS*self.DOWN_TF_FS-self.DOWN_TF_FS//8).astype(np.int64)) for i in range(len(start_ind_valid))])
    noisesample_re = wave_re_spec[baseline_ind_spec]
    noise = 10**(((np.concatenate(noisesample_re_,axis=0).mean(0)+1)/2*(cfg.MODEL.MAX_DB-cfg.MODEL.NOISE_DB)+cfg.MODEL.NOISE_DB)/10)
    ```  